In [130]:
# !git add .
# !git commit -m "Changed DB classification field"
# !git pull
# !git push -u origin main

[main 440484f] Changed DB classification field
 1 file changed, 6 insertions(+), 6 deletions(-)
Already up to date.
branch 'main' set up to track 'origin/main'.


To github.com:thernandez7/Sports_Misinformation_Tool.git
   660e1c2..440484f  main -> main


## Sports Misinformation Classification Tool

### Connect to the Database

In [67]:
import mysql.connector
from sqlalchemy import create_engine
import pandas as pd

username = 'root'  
password = 'password'
host = 'localhost' 
database = 'sports_news_db'

connection = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
    database=database
)

if connection.is_connected():
    print("Connected to the database successfully!")

engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}/{database}", echo=False)

Connected to the database successfully!


#### Use this format to insert into the database

INSERT INTO articles (team_or_player, source, publication_date, content, trust_score, classification, link)
VALUES
('New York Yankees, Los Angeles Lakers', 'ESPN', '2024-10-27', 'Yankees article content example.', 85.00, 'real', 'https://example.com/article1'),
('Los Angeles Lakers', 'Twitter', '2024-10-27', 'Lakers article content example.', 60.00, 'fake', 'https://example.com/article2');


#### Table fields 
Table Name: articles

Fields: 

     id INT AUTO_INCREMENT PRIMARY KEY,
     
     team_or_player VARCHAR(500), (This will be the article title that we can query- usually includes teams or names in it)
     
     source VARCHAR(200),
     
     publication_date DATE,
     
     content TEXT,
     
     trust_score DECIMAL(5, 2), 
     
     classification ENUM('credible', 'uncredible', 'unknown') DEFAULT 'unknown',

     link VARCHAR(255)

### Simulate Tool Working

In [118]:
team_or_player = input("Enter the team or player's name: ")

query = f"SELECT * FROM articles WHERE team_or_player LIKE '%{team_or_player}%'" #search for entered name/team in the title 
df_result = pd.read_sql(query, con=engine, params={'team_or_player': team_or_player})

#get results
if not df_result.empty:
    print(f"Articles related to {team_or_player}:")
    display(df_result)
else:
    print(f"No articles found for {team_or_player}.")

Enter the team or player's name:  x


No articles found for x.


### Get Article Entries from RSS Feeds-- \*not continuous*

In [134]:

# # reddit API - collected 1876 posts
# import csv
# import praw
# from datetime import datetime


# reddit = praw.Reddit(
#     client_id='w-kwRyPigyjYeG9DOiDc8g', 
#     client_secret='ZeDsvNH2YlpVH7F9wEWPkt5wkjLzqA',  
#     user_agent='sports_misinfo_script'  
# )

# subreddit = reddit.subreddit('sports+fantasyfootball') #2 subreddits 

# recent_posts = []
# for post in subreddit.new(limit= 5000):
#     created_date = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
#     recent_posts.append({
#         'title': post.title,
#         'score': post.score,
#         'url': post.url,
#         'id': post.id,
#         'author': str(post.author),
#         'text': post.selftext,
#         'created_date': created_date,
#         'num_comments': post.num_comments,
#         'subreddit': post.subreddit.display_name,

#     })

# print(f"Fetched {len(recent_posts)} posts")

# for i, post in enumerate(recent_posts[:5]):
#     print(f"{i+1}. Title: {post['title']} | Score: {post['score']} | URL: {post['url']}")


# #-------------------------------
# #Save the data to a csv file
# fieldnames = ['title', 'score', 'url', 'id', 'author', 'text', 'created_date', 'num_comments', 'subreddit']

# with open('recent_sports_reddit_posts.csv', mode='w', newline='', encoding='utf-8') as csv_file:
#     writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

#     writer.writeheader()

#     for post in recent_posts:
#         writer.writerow(post)

# print("Data successfully saved to 'recent_sports_reddit_posts.csv'")
# print("Data saved")


In [144]:
import feedparser
import requests
from bs4 import BeautifulSoup

#add urls to this list to parse
url_list = [
        "https://api.foxsports.com/v2/content/optimized-rss?partnerKey=MB0Wehpmuj2lUhuRhQaafhBjAJqaPU244mlTDK1i&size=30&tags=fs/nfl",#fox NFL
        "https://api.foxsports.com/v2/content/optimized-rss?partnerKey=MB0Wehpmuj2lUhuRhQaafhBjAJqaPU244mlTDK1i&size=30", #fox MLB
        "https://api.foxsports.com/v2/content/optimized-rss?partnerKey=MB0Wehpmuj2lUhuRhQaafhBjAJqaPU244mlTDK1i&size=30&tags=fs/nba", #fox NBA
        "https://www.sportscollectorsdaily.com/feed/", #Sports Collectors Daily 
        "https://www.espn.com/espn/rss/news", #ESPN top headlines
        "https://deadspin.com/rss/", #Deadspin 
        "https://news.sportslogos.net/feed/", #SportsLogos.Net
    
]
    
entries = [] #list of dictionaries

#--------------DEFINE FUNCTION TO SCRAPE-------------------
def scrape_article_content(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        #extract content from common tags (e.g., paragraphs, article body)
        article_body = (
            soup.find("article") or
            soup.find("div", {"class": "post-content"}) or
            soup.find("div", class_="article-body") or
            soup.find("div", class_="article-content") or
            soup.find("section", class_="article-section") or
            soup.find("div", class_="main-content") or
            soup.find("div", class_="content-body")
        )
        
        if article_body:
            paragraphs = article_body.find_all("p")
        else:
            paragraphs = soup.find_all("p")

        #join all paragraphs into a single string
        article_content = " ".join(p.get_text() for p in paragraphs)
        return article_content.strip() if article_content else "No content found"

    except Exception as e:
        print(f"Failed to scrape content from {url}: {e}")
        return "Failed to fetch content"




for url in url_list:
    feed = feedparser.parse(url)
    feed_title= feed.feed.title
    
    for entry in feed.entries:
        entry_title= entry.title
        entry_link= entry.link
        entry_published_date= entry.published
        entry_summary= entry.summary
        entry_content = scrape_article_content(entry_link) #scrape
        
        entries.append({
            "feed_title": feed_title,
            "entry_title": entry_title,
            "entry_link": entry_link,
            "entry_published_date": entry_published_date,
            "entry_summary": entry_summary,
            "entry_content": entry_content,
        })

df = pd.DataFrame(entries)
print(df)

df.to_csv('RSS_sports_feeds.csv', index=False)

Failed to scrape content from https://www.espn.com/nfl/story/_/id/42310268/rams-jalen-ramsey-trade-first-domino-team-philosophy-change: 403 Client Error: Forbidden for url: https://www.espn.com/nfl/story/_/id/42310268/rams-jalen-ramsey-trade-first-domino-team-philosophy-change
Failed to scrape content from https://www.espn.com/nfl/story/_/id/42315741/miami-dolphins-tyreek-hill-worst-season-2016: 403 Client Error: Forbidden for url: https://www.espn.com/nfl/story/_/id/42315741/miami-dolphins-tyreek-hill-worst-season-2016
Failed to scrape content from https://www.espn.com/college-football/story/_/id/42329430/wisconsin-adviser-jack-del-rio-arrested-owi-crash: 403 Client Error: Forbidden for url: https://www.espn.com/college-football/story/_/id/42329430/wisconsin-adviser-jack-del-rio-arrested-owi-crash
Failed to scrape content from https://www.espn.com/college-football/story/_/id/42334280/sec-fines-ole-miss-350k-lsu-250k-fan-incidents: 403 Client Error: Forbidden for url: https://www.espn.

In [113]:
#if we want to get contents of whole article we would need to scrape it 
 
 
 
 

### Format DF to match DB table

In [123]:
from datetime import datetime
from dateutil import parser

df['publication_date'] = df['entry_published_date'].apply(lambda x: parser.parse(x).date()) #parse different date formats to date object format
dbdf = pd.read_csv('RSS_sports_feeds.csv')
dbdf['team_or_player'] = df['entry_title']
dbdf['source'] = df['feed_title']
dbdf['publication_date'] = df['publication_date'] 
dbdf['content'] = df['entry_summary']
dbdf['trust_score'] = 0.00  #default
dbdf['classification'] = 'unknown' #default
dbdf['link'] = df['entry_link']

#make a new df in the format of the DB table for easy inserting
sports_DB_df = dbdf[['team_or_player', 'source', 'publication_date', 'content', 'trust_score', 'classification', 'link']]
#save to a new CSV 
sports_DB_df.to_csv('formatted_sports_posts_for_DB.csv', index=False)
print("done")

done


C:\Users\tizia\Anaconda4\Lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


## Define labeling approach to get classification and trust score to update sports_DB_df with ground truths